# Keys & Addresses — Student Edition

**What you'll learn (in plain English):**
- What private and public keys are (on the same curve Bitcoin/Ethereum use)
- How hashes work (SHA‑256 vs Keccak‑256)
- How to make a **Bitcoin legacy address** (starts with `1…`)
- How to make an **Ethereum address** (starts with `0x…`)
- How checksums catch typos

**Keep it simple:** we use small helper functions and common libraries so you can focus on concepts, not low‑level details.

In [6]:
# (Run this once) Install the small libraries we use
# If this fails in your environment, you may already have them.
# Remove the leading % if running outside Jupyter.
%pip install ecdsa eth-utils eth-keys 'eth-hash[pycryptodome]' base58 -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'eth-hash[pycryptodome]'": Expected package name at the start of dependency specifier
    'eth-hash[pycryptodome]'
    ^


## 1) Private key → Public key (same curve for BTC & ETH)
- A **private key** is just a random 32‑byte number.
- A **public key** is derived from it using elliptic‑curve math (secp256k1).
- Public keys can be shown **uncompressed (65 bytes)** or **compressed (33 bytes)**.

👉 We'll generate a keypair and show both public‑key formats.

In [8]:
from ecdsa import SigningKey, SECP256k1
import os

# Generate a random 32-byte private key
priv = os.urandom(32)
sk = SigningKey.from_string(priv, curve=SECP256k1)
vk = sk.verifying_key

# Build uncompressed and compressed public keys
x = vk.pubkey.point.x(); y = vk.pubkey.point.y()
x_bytes = x.to_bytes(32, 'big'); y_bytes = y.to_bytes(32, 'big')
pub_uncompressed = b'\x04' + x_bytes + y_bytes
pub_compressed = (b'\x02' if (y % 2 == 0) else b'\x03') + x_bytes

print('Private key (hex):', priv.hex())
print('Public key (uncompressed, 65B):', pub_uncompressed.hex())
print('Public key (compressed, 33B):', pub_compressed.hex())

Private key (hex): 0db8c11988985cb23539f7e82108da86343a03dcf6cb9ccaa763b52911ce58b8
Public key (uncompressed, 65B): 048b7700bdd3d1465fc1730fc2ec5bbcbfa6bca6a228774130928d624ab76caa0a4bd2a240036bad9b6bad827bee6186fb8e37dc21e79b0145d8c13cf512b4bc80
Public key (compressed, 33B): 028b7700bdd3d1465fc1730fc2ec5bbcbfa6bca6a228774130928d624ab76caa0a


## 2) Hash functions (SHA‑256 vs Keccak‑256)
- A **hash** turns any input into a fixed‑size output.
- **Bitcoin** uses **SHA‑256** a lot.
- **Ethereum** uses **Keccak‑256** (note: this is *not* the same as `hashlib.sha3_256`).

In [9]:
import hashlib
from eth_utils import keccak

data = b'hello world'
print('SHA-256   :', hashlib.sha256(data).hexdigest())
print('SHA3-256  :', hashlib.sha3_256(data).hexdigest())
print('Keccak-256:', keccak(data).hex())

SHA-256   : b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9
SHA3-256  : 644bcc7e564373040999aac89e7622f3ca71fba1d972fd94a31c3bfbf24e3938
Keccak-256: 47173285a8d7341e5e972fc677286384f802f8ef42a5ec5f03bbfa254cb01fad


## 3) Bitcoin legacy address (P2PKH, starts with `1…`)
Formula (don’t memorize — just understand the flow):
1. `HASH160(pubkey)` = `RIPEMD‑160(SHA‑256(pubkey))` → 20 bytes
2. Prepend **version byte** `0x00` for mainnet (testnet uses `0x6F`)
3. Base58Check‑encode the result (this adds a 4‑byte checksum automatically)

👉 We'll make an address from the **compressed** pubkey we generated.

In [10]:
import hashlib, base58

def hash160(b: bytes) -> bytes:
    return hashlib.new('ripemd160', hashlib.sha256(b).digest()).digest()

def btc_p2pkh_address(pubkey_compressed: bytes, mainnet: bool = True) -> str:
    vh160 = (b'\x00' if mainnet else b'\x6F') + hash160(pubkey_compressed)
    return base58.b58encode_check(vh160).decode()

btc_addr = btc_p2pkh_address(pub_compressed, mainnet=True)
print('Bitcoin P2PKH address:', btc_addr)

Bitcoin P2PKH address: 1BB2N3h2Msz4gcsqUqeEKn6W8xk9BboJYV


## 4) Export the private key as WIF (Wallet Import Format)
- WIF packs the private key + a flag for “compressed pubkey” + checksum.
- Mainnet WIF uses version byte `0x80`.
- If your pubkey is compressed, append `0x01` before encoding.

👉 Most wallets accept WIF so you can import this key easily (on testnets only for practice).

In [11]:
def btc_wif_from_priv(privkey: bytes, compressed: bool = True, mainnet: bool = True) -> str:
    version = b'\x80' if mainnet else b'\xEF'
    payload = version + privkey + (b'\x01' if compressed else b'')
    return base58.b58encode_check(payload).decode()

wif = btc_wif_from_priv(priv, compressed=True, mainnet=True)
print('WIF (keep secret!):', wif)

WIF (keep secret!): KwgPEFZw8iwsXVetCdyeDw86fX9AgasxoWNTjcyZoy1fWBdvC21z


## 5) Ethereum address (starts with `0x…`)
Steps:
1. Take the **uncompressed** public key **without** the `0x04` prefix (so 64 bytes = X||Y).
2. `Keccak‑256` it and keep the **last 20 bytes**.
3. Apply the **EIP‑55** mixed‑case checksum for easier reading.

👉 We'll generate it from the same private key.

In [12]:
from eth_keys import keys
from eth_utils import keccak, to_checksum_address

eth_sk = keys.PrivateKey(priv)
eth_vk = eth_sk.public_key
pubkey_64 = eth_vk.to_bytes()  # 64 bytes X||Y
addr_bytes = keccak(pubkey_64)[-20:]
addr_hex = '0x' + addr_bytes.hex()
addr_eip55 = to_checksum_address(addr_hex)

print('Ethereum (no checksum):', addr_hex)
print('Ethereum (EIP-55)    :', addr_eip55)

Ethereum (no checksum): 0xa2a374922196562c4aa85a8b3a527508820132f3
Ethereum (EIP-55)    : 0xA2a374922196562c4aa85a8B3A527508820132F3


## 6) Sign & verify a message (minimal)
- **Bitcoin-style demo:** sign the SHA‑256 of a message using ECDSA.
- **Ethereum-style demo:** sign the Keccak‑256 of a message.

*(This is a teaching demo. Real wallets also add prefixes like `"\x18Ethereum Signed Message:\n…"` before hashing.)*

In [13]:
import hashlib
from ecdsa import BadSignatureError

msg = b'hello class'

# Bitcoin-ish (hash with SHA-256)
btc_msg_hash = hashlib.sha256(msg).digest()
sig_der = sk.sign_deterministic(btc_msg_hash)
print('ECDSA signature (DER hex):', sig_der.hex())
print('Verify (SHA-256) OK?:', sk.verifying_key.verify(sig_der, btc_msg_hash))

# Ethereum-ish (hash with Keccak-256)
eth_msg_hash = keccak(msg)
eth_signature = eth_sk.sign_msg_hash(eth_msg_hash)
print('Ethereum signature (r||s||v hex):', eth_signature.to_bytes().hex())
print('Verify (Keccak-256) OK?:', eth_sk.public_key.verify_msg_hash(eth_msg_hash, eth_signature))

ECDSA signature (DER hex): 7587ab5e29dfdcc97ba0ff06d8a4c4be9dc2f857055e3df1f2a97373b146786b78f6374d38f1172ed0849c719c88c0db90ca6b3b6ce0bf4c3f896fe34eb8b108
Verify (SHA-256) OK?: True
Ethereum signature (r||s||v hex): e64b48230ac0862a83077cbece5ed697d2369891df39a0ccfb6c9a858ba6cc9146194330da597acdafcce0c766993faf858de1e9737156fd12fad60f49e1083400
Verify (Keccak-256) OK?: True


## 7) Checksums catch typos
- Base58Check has a 4‑byte checksum. If you mistype an address, decoding should fail.
- EIP‑55 uses **mixed‑case** to detect many common mistakes in Ethereum addresses.

In [14]:
# Try corrupting one character of the Bitcoin address and decode
bad_btc = btc_addr[:5] + ('1' if btc_addr[5] != '1' else '2') + btc_addr[6:]
try:
    base58.b58decode_check(bad_btc)
    print('Unexpectedly valid (rare).')
except Exception as e:
    print('Checksum caught the error:', e)

# Try changing letter case in the checksummed ETH address (often breaks validation in wallets)
print('Original EIP-55:', addr_eip55)
tweaked = '0x' + addr_eip55[2:].swapcase()
print('Tweaked case   :', tweaked)
print('Note: Proper validators will reject the wrong casing.')

Checksum caught the error: Invalid checksum
Original EIP-55: 0xA2a374922196562c4aa85a8B3A527508820132F3
Tweaked case   : 0xa2A374922196562C4AA85A8b3a527508820132f3
Note: Proper validators will reject the wrong casing.


## Practice (do these on your own)
1. Generate **three** new keypairs. Make Bitcoin & Ethereum addresses for each.
2. Export each private key as **WIF** and then import into a testnet wallet (never mainnet for class demos).
3. Change **one character** in each Bitcoin address and confirm that Base58Check decoding fails.
4. Take any Ethereum address, remove `0x`, lowercase it, and rebuild the **EIP‑55** version.
5. (Stretch) Read about Bech32 (SegWit) and try making a `bc1…` address using a library.